# Introduction
<a id="introduction"></a>

Here are two main targets in this assignment. 

First is to integrate several datasets into one single schema and input some data and fix possible problems in this final data.

Second is to find which attributes( Rooms, crime_C_average,travel_min_to_CBD, and property_age) is important to predict the price when we want to build a linear model. In addition, in this part we will use different normalization/ transformation metods, such as standardization, min-max normalization, log, power, and root transformation on attributes to observe and explore the effect on the linear model we want to build.



# Import libraries
<a id="importlibraries"></a>
Import some libraries for this assignment:
* pandas
* re 
* numpy 
* sklearn
* BeautifulSoup
* geopandas
* pygtfs
* shapely
* math
* datetime
* urllib
* difflib
* zipfile

In [1]:
!pip install pandas
!pip install bs4
!pip install geopandas
!pip install pygtfs


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


# Talbe of contents

* [Introduction](#introduction)
* [Import libraries](#importlibraries)
* [Task 1: Data Integration](#task_1)
    * [Step 01: Find the suburb](#step01)
    * [Step 02: Find the closest train station id and distance(m)](#step02)
    * [Step 03: Calculate the average travel time from the closest train station to Southern Cross Station](#step03)
    * [Step 04: over priced?](#step04)
    * [Step 05: crime A/B/C average](#step05)
    * [Step 06: Find the closest primary/ Secondary school and the distance(m) between property](#step06)
    * [Step 07: Primary / Secondary School Ranking](#step07)

* [Taks 2: Data reshaping](#task_2)

* [Summary](#summary)

In [2]:
import pandas as pd
crimebylocationdatatable_yearending31march2016 = pd.read_excel('crimebylocationdatatable-yearending31march2016.xlsx', index_col=0)
crimebylocationdatatable_yearending31march2016.columns = crimebylocationdatatable_yearending31march2016.iloc[0]
crimebylocationdatatable_yearending31march2016 = crimebylocationdatatable_yearending31march2016[1:]
crimebylocationdatatable_yearending31march2016

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [ ]:
import pygtfs
from bs4 import BeautifulSoup as bsoup
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape, Point
import fiona
from math import *
import numpy as np
from datetime import datetime, timedelta
from urllib.request import urlopen
import re
import difflib
from zipfile import ZipFile
import os
import sys 
from sklearn import preprocessing
from matplotlib import pyplot as plt
import statsmodels.formula.api as sm
import math

# Task 1: Data Integration
<a id="task_1"></a>
In this task, integrate some datasets into on with the following schema.

In [ ]:
#read the csv file
df = pd.read_csv("Group081.csv")

## Find the suburb
<a id="step01"></a>

The target in this step is to find the property in which suburb.

We use the Vic_suburb_boundary.zip file to set the suburb boundary, and if the property's lattitude and longtitude are included in the suburb's lattitude and longtitude, we fill the suburb for this property.

In [ ]:
# Extract the zip file of vic_suburb_boundary.zip
zip_boundary = ZipFile('vic_suburb_boundary.zip')
zip_boundary.extractall()

In [ ]:
# open the VIC_LOCALITY_POLYGON_shp.shp by using fiona libary
collection = fiona.open('VIC_LOCALITY_POLYGON_shp.shp')

shapes = {}
townnames = {}
for f in collection:
    # the surburb id
    town_id = f['properties']['LC_PLY_PID']  
    # the lattitude and longtitude boundary of surburb
    shapes[town_id] = shape(f['geometry']) 
    # the surburb name
    townnames[town_id] =f['properties']['VIC_LOCA_2']
    
def search(lat, long):  
    global shapes, townnames
    #if the lattitude and longtitude are included in the suburb's lattitude and longtitude, return suburb name.
    return next((townnames[town_id]  for town_id in shapes if shapes[town_id].contains(Point(lat, long))), None)

In [ ]:
# apply the search hunction to the df
df['Suburb']=df.apply(lambda x: search(x.Longtitude, x.Lattitude),axis=1)

## Find the train station id and station distance (m)
<a id="step02"></a>
The target in this part is to find the closest train staion to the property which has a direct trip to the Southern Cross Railway Station and the direct distance from the closest train station to the property.

In this section, we first extract gtfs file to find the stations and trips data, and then create get distance function to calculate the distance from the property to the cloest train station.



In [ ]:
#Extract the gtfs file
zip_gtfs = ZipFile('gtfs.zip')
zip_gtfs.extractall()

In [ ]:
# create a schedule object (a sqlite database)
sched = pygtfs.Schedule(":memory:")                
# append data to schedule object
pygtfs.append_feed(sched, "./1/google_transit.zip")
pygtfs.append_feed(sched, "./2/google_transit.zip")

#### In this step, we find the station name and station's Longtitude and Lattitude.

In [ ]:
# get the query of trips in schedule
sched_trips=sched.trips_query
# get the query of services in schedule
sched_services=sched.services_query
# get the query of stops in schedule
sched_stops=sched.stops_query
# get the query of stop_times in schedule
sched_stop_time=sched.stop_times_query.all()



#stops={}
stops_id=[]
station_name=[]
station_lat_lon=[]

# for loop in the query of stops
for i in sched_stops.all():
    stop_id=i.stop_id
    stop_name=i.stop_name
    stop_lat=i.stop_lat
    stop_lon=i.stop_lon
    #stops[stop_id]={'stop_name':stop_name,'stop_lat':stop_lat,'stop_lon':stop_lon}
    lat_lon=[stop_lat,stop_lon]
    #append stop name to station_name list
    
    if stop_id not in stops_id:
        station_name.append(stop_name)
        #append lat and long to lat_lon list
        station_lat_lon.append(lat_lon)
        #append stop id to stop_id list
        stops_id.append(stop_id)

# translate the list of station_lat_lon to matrix
station_lat_lon_mat=np.matrix(station_lat_lon)   
    

#### In this step, we find the service ids that are work in weekday.

In [ ]:
# get the query of services in schedule
sched_service=sched.services_query.all()
weekday_service_id=[]

#for loop in sched_service
for i in sched_service:
    service_id=i.service_id
    M=int(i.monday)
    T=int(i.tuesday)
    W=int(i.wednesday)
    Th=int(i.thursday)
    F=int(i.friday)
    
    #if M+T+W+Th+F >=1, append service_id to weekday_service_id
    if M+T+W+Th+F == 5 and service_id not in weekday_service_id:
        weekday_service_id.append(service_id)

#### In this step, we filter trips that are work in weekday.

In [ ]:
weekday_trip=[]
#for loop in the query of services in schedule
for i in sched_trips.all():
    route_id=i.route_id
    service_id=i.service_id
    trip_id=i.trip_id
    #if service_id in weekday_service_id, append trip_id to weekday_trip
    if service_id in weekday_service_id:
        weekday_trip.append(trip_id)

#### In this step, we find trips that are work in weekday and departrue time between 8:00 am to 9:30am and combine with the station id.

In [ ]:
southern_cross_stop_id=['20043','22180']
# set a dictionary to store the trip and stop station id
trip_stop={}

# for loop in the query of stop_time
for i in sched_stop_time:
    stop_list=[]
    stop_id=i.stop_id
    trip_id=i.trip_id
    # append stop_id to stop_list
    stop_list.append(stop_id)
    arrival_time=i.arrival_time
    departure_time=i.departure_time
    # if departure_time between '7:00:00' and '9:30:00', trip_id in trip_list, stop_id equals to station_id, append departure_time to departure_time_list
    if str(departure_time) >= '7:00:00' and str(departure_time) < '9:30:00' and trip_id in weekday_trip: 
        if trip_id not in trip_stop.keys():
            trip_stop[trip_id]=stop_list
        else:
            original_value=trip_stop[trip_id]
            original_value.append(stop_id)
            trip_stop[trip_id]=original_value
    # if trip_id in trip_stop.keys and if stop_id equals to southern_corss_stop_id, add the value to trip_stop[trip_id]
    elif trip_id in trip_stop.keys():
        if stop_id == southern_cross_stop_id[0]:
            original_value=trip_stop[trip_id]
            original_value.append(southern_cross_stop_id[0])
            trip_stop[trip_id]=original_value
        elif stop_id == southern_cross_stop_id[1]:
            original_value=trip_stop[trip_id]
            original_value.append(southern_cross_stop_id[1])
            trip_stop[trip_id]=original_value
            

#### Create the get_distance function that calculate the distance between property and station.

In [ ]:
def get_distance(target,house):
    # radius of equator: KM
    ra = 6378  
    
    # change angle to radians
    target_rad_lat = np.radians(target[:,0])
    target_rad_lon = np.radians(target[:,1])
    house_rad_lat = np.radians(house[:,0])
    house_rad_lon = np.radians(house[:,1])
    
    dlon = house_rad_lon - target_rad_lon
    dlat = house_rad_lat - target_rad_lat
    a=np.power((np.sin(dlat/2)),2)+ np.multiply(np.multiply(np.cos(target_rad_lat),np.cos(house_rad_lat)),np.power((np.sin(dlon/2.0)),2))
    c = 2 * np.arcsin(np.sqrt(a))
    distance = ra * c *1000
    
    return distance

#### Create the find_station function to find the closest station and the distance between this station.

In [ ]:
def find_station_name(lat,lon):
    Find_loc = np.matrix([[lat,lon]])
    #run get_distance function to get all distance between station.
    all_distance = get_distance(station_lat_lon_mat,Find_loc)
    #list 100 lowest distance
    rank_100=list(all_distance.argsort(axis=0)[:100])
    #for loop in rank_100
    for i in rank_100:
        # if station id equals to southern_cross_stop_id, return southern_cross_stop_id, and break the function
        if stops_id[int(i)] == southern_cross_stop_id[0]:
            return southern_cross_stop_id[0]
            break
        elif stops_id[int(i)] == southern_cross_stop_id[1]:
            return southern_cross_stop_id[1]
            break
            
        else:
            #for loop in the values in the dictionary of trip_stop 
            for j in trip_stop.values(): 
                # if target station id and southern_corss_station id all in trip, return station name and distance
                if stops_id[int(i)] in j and southern_cross_stop_id[0] in j and j.index(stops_id[int(i)]) < j.index(southern_cross_stop_id[0]):
                    station=stops_id[int(i)]
                    distance=round(float(all_distance[int(i)]),3)
                    return station
                    break
                elif stops_id[int(i)] in j and southern_cross_stop_id[1] in j and j.index(stops_id[int(i)]) < j.index(southern_cross_stop_id[1]):
                    station=stops_id[int(i)]
                    distance=round(float(all_distance[int(i)]),3)
                    return station
                    break

#### Create the find_station function to find the closest station and the distance between this station.

In [ ]:
def find_station_distance(lat,lon):
    Find_loc = np.matrix([[lat,lon]])
    #run get_distance function to get all distance between station.
    all_distance = get_distance(station_lat_lon_mat,Find_loc)
    #list 500 lowest distance
    rank_10=list(all_distance.argsort(axis=0)[:100])
    for i in rank_10:
        for j in trip_stop.values(): 
            # if target station id and southern_corss_station id all in trip, return station name and distance
            if stops_id[int(i)] in j and (southern_cross_stop_id[0] in j or southern_cross_stop_id[1] in j):
                station=stops_id[int(i)]
                distance=round(float(all_distance[int(i)]),3)
                return distance
                break

In [ ]:
# apply the find_station_name function to df
df['train_station_id']=df.apply(lambda x: find_station_name(x.Lattitude, x.Longtitude),axis=1)
# apply the find_station_distance function to df
df['distance_to_train_station']=df.apply(lambda x: find_station_distance(x.Lattitude, x.Longtitude),axis=1)

## travel_min_to_CBD
<a id="step03"></a>

In this target, we will find the average travel time(minutes) from the closest train station to the Southern Cross Railway Station on weekdays.


#### Create the average_min function to calcualte the average travel time(minutes) from the closest train station to the Southern Cross Railway Station on weekdays

In [ ]:
def average_min(station_id):
    station_id=station_id
    if station_id not in southern_cross_stop_id:
        trip_list=[]
        #for loop in trip_stop dictionary
        for key,values in trip_stop.items():
            # if station_id and southern_cross_stop_id all in values, key in weekday_trip,and the index of station_id less than southern_cross_stop_id in values, append key to trip_list
            if station_id in values and southern_cross_stop_id[0] in values and values.index(station_id)<values.index(southern_cross_stop_id[0]):
                if key in weekday_trip:
                    trip_list.append(key)
            # if station_id and southern_cross_stop_id all in values, key in weekday_trip,and the index of station_id less than southern_cross_stop_id in values, append key to trip_list
            elif station_id in values and southern_cross_stop_id[1] in values and values.index(station_id)<values.index(southern_cross_stop_id[1]):
                if key in weekday_trip:
                    trip_list.append(key)
    
    
        departure_time_list=[]
        arrival_time_list=[]
    
        #for loop in sched_stop_time
        for i in sched_stop_time:
            stop_id=i.stop_id
            trip_id=i.trip_id
            arrival_time=i.arrival_time
            departure_time=i.departure_time
            # if trip_id in trip_list, stop_id equals to station_id, append departure_time to departure_time_list
            if trip_id in trip_list and stop_id == station_id: 
                departure_time_list.append(departure_time)
            #if trip_id in trip_list and stop_id equals to southern_cross_stop_id, append arrival_time to arrival_time_list
            if trip_id in trip_list and stop_id in southern_cross_stop_id: 
                arrival_time_list.append(arrival_time)
            
        #calculate the average by minus arrival_time and departure_time
        sum_second=0
        count=0
        for i in range(len(arrival_time_list)):
            sum_second+=(arrival_time_list[i]-departure_time_list[i]).seconds
            count+=1

        average_min=(sum_second/60)/count
    
    #if the station_id is equals to southern cross station, the average_min equals to zero
    else:
        average_min=0
    
    #return average_min
    return average_min


In [ ]:
df['travel_min_to_CBD']=df.apply(lambda x: average_min(x.train_station_id),axis=1)

## over priced?
<a id="step04"></a>
In this step, we will find the boolean feature that whether or not the price of the property is higher than the median price of similar properties( with same bedrooms, batheroom, parking_space, and property_type) in the same suburb on the year of selling.


In [ ]:
df['Year'] = df.apply(lambda x: x.Date[-4:], axis=1) # append a new column which stands for the year of selling.

In [ ]:
# create a list storing columns used to check similarity
similar_list = ['Year',
                'Rooms',
                'Type',
                'Bathroom',
                'Car',
                'Suburb']

# get mean
df_mean = df.groupby(similar_list).mean()

# get count
df_count = df.groupby(similar_list).count()

#### define a function to get key for df_mean and df_count

In [ ]:
def get_key(x):
    key = (x.Year, x.Rooms, x.Type, x.Bathroom, x.Car, x.Suburb)
    return key

# create a new column named 'over_priced', and if there is no any similar property
df['over_priced?'] = df.apply(lambda x: x.Price > df_mean.loc[get_key(x)].Price,
                              axis=1)

#### open councils.txt and create a dictionary which takes suburbs as keys and local government area as values

In [ ]:
with open('councils.txt') as councils:
    sub_dict = {}
    
    for line in councils:
        # spilt line to a list
        line_list = line.strip().split(' :')
        
        # area means the loval government area
        area = line_list[0]
        
        # transform the suburbs string into a list
        sub_list = re.findall(r'\'(.*?)\'', line_list[1])
        
        # add new items into the dictionary
        for suburb in sub_list:
            sub_dict[suburb.upper()] = area

## Crime A/B/C average
<a id="step05"></a>

In this step, we calculate the average of type A/B/C crime in the local government area that the property belongs to , in the three years prior to selling the property as the property.

In [ ]:
# open crimebylocationdatatable-yearending31march2016.xlsx, starts from 20th row
xl = pd.read_excel('crimebylocationdatatable-yearending31march2016.xlsx',
                   sheet_name='Table 1',
                   skiprows=19)

# drop an useless column
crime = xl.drop(columns='Unnamed: 0')

In [ ]:
# group by year, crime type and government area and get summary of each group
crime_sum = crime.groupby(['Apr - Mar reference period',
                           'CSA Offence Division',
                           'Local Government Area']).sum()

#### define functions to get the average of crimes in the three years prior to selling the property.

In [ ]:
def get_A(x):
    
    # check if the key is in the crime_sum
    if (int(x.Year)-1, 'A Crimes against the person', sub_dict[x.Suburb]) in crime_sum.index\
    and (int(x.Year)-2, 'A Crimes against the person', sub_dict[x.Suburb]) in crime_sum.index\
    and (int(x.Year)-3, 'A Crimes against the person', sub_dict[x.Suburb]) in crime_sum.index:
        
        # get the summary of three years
        # keys are year, crime type and government area
        sum = \
        crime_sum.loc[(int(x.Year)-1, 'A Crimes against the person', sub_dict[x.Suburb])]['Offence Count'] +\
        crime_sum.loc[(int(x.Year)-2, 'A Crimes against the person', sub_dict[x.Suburb])]['Offence Count'] +\
        crime_sum.loc[(int(x.Year)-3, 'A Crimes against the person', sub_dict[x.Suburb])]['Offence Count']
        
        # get the average
        avg = sum/3
        
    # if the year of selling is earlier than 2015, set it to default value: -1 
    else:
        avg = -1
    return avg

In [ ]:
def get_B(x):
    if (int(x.Year)-1, 'B Property and deception offences', sub_dict[x.Suburb]) in crime_sum.index\
    and (int(x.Year)-2, 'B Property and deception offences', sub_dict[x.Suburb]) in crime_sum.index\
    and (int(x.Year)-3, 'B Property and deception offences', sub_dict[x.Suburb]) in crime_sum.index:
        sum = \
        crime_sum.loc[(int(x.Year)-1, 'B Property and deception offences', sub_dict[x.Suburb])]['Offence Count'] +\
        crime_sum.loc[(int(x.Year)-2, 'B Property and deception offences', sub_dict[x.Suburb])]['Offence Count'] +\
        crime_sum.loc[(int(x.Year)-3, 'B Property and deception offences', sub_dict[x.Suburb])]['Offence Count']
        avg = sum/3
    else:
        avg = -1
    return avg

In [ ]:
def get_C(x):
    if (int(x.Year)-1, 'C Drug offences', sub_dict[x.Suburb]) in crime_sum.index\
    and (int(x.Year)-2, 'C Drug offences', sub_dict[x.Suburb]) in crime_sum.index\
    and (int(x.Year)-3, 'C Drug offences', sub_dict[x.Suburb]) in crime_sum.index:
        sum = \
        crime_sum.loc[(int(x.Year)-1, 'C Drug offences', sub_dict[x.Suburb])]['Offence Count'] +\
        crime_sum.loc[(int(x.Year)-2, 'C Drug offences', sub_dict[x.Suburb])]['Offence Count'] +\
        crime_sum.loc[(int(x.Year)-3, 'C Drug offences', sub_dict[x.Suburb])]['Offence Count']
        avg = sum/3
    else:
        avg = -1
    return avg

In [ ]:
# apply functions for each crime type
df['crime_A_average'] = df.apply(lambda x: get_A(x), axis=1)
df['crime_B_average'] = df.apply(lambda x: get_B(x), axis=1)
df['crime_C_average'] = df.apply(lambda x: get_C(x), axis=1)

In [ ]:
df = df.drop(columns='Year')

## Find the closest primary/ Secondary school and the distance (m) between property
<a id="step06"></a>

In this section, we find the colos the closest primary school to the property. In addition, we will calculate the direct distance between the property and the closest primary school.

We will use bsoup to parse the xml file and extract the data we need, and then use get_distance function to calculate the distance.

In [ ]:
# use bsoup to open the schools.xml file
soup = bsoup(open('schools.xml'),"lxml-xml")
school_name_list=[]
p_school_list=[]
p_school_lat_lon=[]

s_school_list=[]
s_school_lat_lon=[]

# for loop in soup.find_all('school')
for i in soup.find_all('school'):
        School_name=i.find("School_Name").string 
        Education_Sector=i.find("Education_Sector").string 
        Entity_Type=i.find("Entity_Type").string 
        School_No=i.find("School_No").string 
        School_Type=i.find("School_Type").string 
        Full_Phone_No=i.find("Full_Phone_No").string 
        LGA_ID=i.find("LGA_ID").string 
        LGA_Name=i.find("LGA_Name").string 
        X=float(i.find("X").string)
        Y=float(i.find("Y").string)
        school_name_list.append(School_name)
        #if school_type equals to 'Primary' or 'Pri/Sec', append school_name to p_school_list and append lat_lon to p_school_lat_lon
        if School_Type == 'Primary' or School_Type == 'Pri/Sec':
            p_school_list.append(School_name)
            lat_lon=[Y,X]
            p_school_lat_lon.append(lat_lon)
        #if school_type equals to 'Secondary' or 'Pri/Sec', append school_name to s_school_list and append lat_lon to s_school_lat_lon
        if School_Type == 'Secondary' or School_Type == 'Pri/Sec':
            s_school_list.append(School_name)
            lat_lon=[Y,X]
            s_school_lat_lon.append(lat_lon)
            
# translate the list of p_school_lat_lon_matrix to matrix            
p_school_lat_lon_matrix=np.matrix(p_school_lat_lon)  
# translate the list of s_school_lat_lon_matrix to matrix    
s_school_lat_lon_matrix=np.matrix(s_school_lat_lon)  

#### Create the primary_school)name function to find the closest primary school.

In [ ]:
def primary_school_name(lat,lon):
    #matrix the property lat and lon
    Find_loc = np.matrix([[lat,lon]])
    #run get_distance function to get all distance between propery and primary school.
    all_distance = get_distance(p_school_lat_lon_matrix,Find_loc)
    school_name = p_school_list[int(all_distance.argmin(axis=0))]
    distance = round(float(all_distance[int(all_distance.argmin(axis=0))]),3)
    
    return school_name

#### Create the primary_school_distance function to find the closest primary school and the distance between it.

In [ ]:
def primary_school_distance(lat,lon):
    #matrix the property lat and lon
    Find_loc = np.matrix([[lat,lon]])
    #run get_distance function to get all distance between propery and primary school.
    all_distance = get_distance(p_school_lat_lon_matrix,Find_loc)
    school_name = p_school_list[int(all_distance.argmin(axis=0))]
    distance = round(float(all_distance[int(all_distance.argmin(axis=0))]),3)
    
    return distance

#### Create the secondary_school)name function to find the closest secondary school and the distance between it.

In [ ]:
def secondary_school_name(lat,lon):
    Find_loc = np.matrix([[lat,lon]])
    #run get_distance function to get all distance between property and secondary school.
    all_distance = get_distance(s_school_lat_lon_matrix,Find_loc)
    school_name = s_school_list[int(all_distance.argmin(axis=0))]
    distance = round(float(all_distance[int(all_distance.argmin(axis=0))]),3)
    
    return school_name

#### Create the secondary_school_distance function to find the closest secondary school and the distance between it.

In [ ]:
def secondary_school_distance(lat,lon):
    Find_loc = np.matrix([[lat,lon]])
    #run get_distance function to get all distance between property and secondary school.
    all_distance = get_distance(s_school_lat_lon_matrix,Find_loc)
    school_name = s_school_list[int(all_distance.argmin(axis=0))]
    distance = round(float(all_distance[int(all_distance.argmin(axis=0))]),3)
    
    return distance

In [ ]:
#apply primary_school_name function to df
df['closest_primary_school']=df.apply(lambda x: primary_school_name(x.Lattitude, x.Longtitude),axis=1)
#apply primary_school_distance function to df
df['distance_to_closest_primary']=df.apply(lambda x: primary_school_distance(x.Lattitude, x.Longtitude),axis=1)
#apply secondary_school_name function to df
df['closest_secondary_school']=df.apply(lambda x: secondary_school_name(x.Lattitude, x.Longtitude),axis=1)
#apply secondary_school_distance function to df
df['distance_to_closest secondary']=df.apply(lambda x: secondary_school_distance(x.Lattitude, x.Longtitude),axis=1)

## Primary / Secondary School Ranking
<a id="step07"></a>

In this step, we input the rank of the closest primary/ secondary to the propery.

The rank we are using is from web, therefore, we need to use Web Scraping to get the rank which we want.


### Primary

In [ ]:
#use the urlopen() function to open the web
html = urlopen("http://www.schoolcatchment.com.au/?p=12301")
bsobj = bsoup(html, "lxml")

In [ ]:
primary_school_rank={}

# for loop in bsobj.find_all('table')
for i in bsobj.find_all('table'):
    School_rank=i.find_all("td",{'class':'column-1'})
    School_name=i.find_all("td",{'class':'column-2'})    
    # for loop in zip(School_name, School_rank)
    for name, rank in zip(School_name, School_rank):
        #if name not in the dictionary of primary_school_rank, set the value to rank
        if name.string not in primary_school_rank.keys():
            primary_school_rank[name.string]= int(rank.string)
     

#### In this step, we will to check if there are some mismatch between school rank and p_school_list

In [ ]:
wrong_primary=[]
for i in primary_school_rank.keys():
    if i not in p_school_list:
        wrong_primary.append(i)
        print(i)


#### we could notice that there are some mistake in school rank, therefor, we use get_close_matches function to fix the problems

In [ ]:
correct_primary_dict={}

#for loop in wrong_primary
for i in wrong_primary:
    school=difflib.get_close_matches(i, p_school_list, n=1, cutoff=0.90)
    # if the length of school better than zero, set the wrong school name as key and correct school name as value 
    if len(school) >0:
        for j in school:
            correct_primary_dict[i]=j

#for loop to change the wrong school name to correct school name         
for i in correct_primary_dict.keys():
    if i in primary_school_rank.keys():
        primary_school_rank[correct_primary_dict[i]]=primary_school_rank[i]
        del primary_school_rank[i]

#### Create the primary_rank function to input the primary school rank

In [ ]:
def primary_rank(primary_school):
    primary_school_name = primary_school
    # if primary_school_name in the keys in primary_school_rank,set the value in the dictionary of primary_school_rank as rank 
    if primary_school_name in primary_school_rank.keys():
        rank = primary_school_rank[primary_school_name]
    else:
        rank = 'not ranked'
    
    return rank

In [ ]:
#apply primary_rank function to df
df['primary_school_ranking']=df.apply(lambda x: primary_rank(x.closest_primary_school),axis=1)

### Secondary

In [ ]:
secondary_school=[]
text=[]
# input the file
temp = open('Temp.txt', 'w')

# set the identify patterns
pattern_name=re.compile('^<div>([a-zA-Z].*?)<')

#open the 'secondary-school-ranking.htm' file
with open('secondary-school-ranking.htm') as fp:
    # read each line in file
    for line in fp:
        #if line is not end with '</div><hr />\n', '<hr />\n', or '<hr /></ul>\n' replace \n to ' '
        if line != '</div><hr />\n' or line !='<hr />\n' or line !='<hr /></ul>\n':
            line = line.replace('\n',' ')
            line = line.replace('N/A','')
            temp.write(line)
        else:
            temp.write(line)                
        
        #strip the line
        lines=line.strip()
        #append lines to text list
        text.append(lines)

# read each line in text        
for line in text:
    search_name = pattern_name.search(line)
    #if search_name is not None append name to secondary_school
    if search_name is not None:
        name=search_name.group(1)
        if name != 'N/A':
            secondary_school.append(name)

#close the file
temp.close()      

In [ ]:
# input the file
temp = open('Temp.txt', 'r')
rank_list=[]

# set the identify patterns
pattern_rank=re.compile('<div>[a-zA-Z].*?</div> <div>(\d.*?)<')

# read each line in temp
for line in temp:    
    find_rank = pattern_rank.findall(line)
    # for loop in find_rank, and append rank to rank_list
    for rank in find_rank:
        rank_list.append(rank)

#close the file       
temp.close() 

In [ ]:
Secondary_school_rank={}
#for loop in zip(secondary_school,rank_list)
for school,rank in zip(secondary_school,rank_list):
    # if school not in secondary_school_rank, set school as key, rank as value in dictionary of Secondary_school_rank
    if school not in Secondary_school_rank.keys():
        Secondary_school_rank[school]=rank

#### In this step, we will to check if there are some mismatch between school rank and s_school_list

In [ ]:
wrong_Secondary=[]
for i in Secondary_school_rank.keys():
    if i not in s_school_list:
        wrong_Secondary.append(i)
        print(i)

#### we could notice that there are some mistake in school rank, therefor, we use get_close_matches function to fix the problems

In [ ]:
correct_Secondary_dict={}

#for loop in wrong_secondary
for i in wrong_Secondary:
    school=difflib.get_close_matches(i, s_school_list, n=1, cutoff=0.85)
    # if the length of school better than zero, set the wrong school name as key and correct school name as value 
    if len(school) >0:
        for j in school:
            correct_Secondary_dict[i]=j
            
#for loop to change the wrong school name to correct school name
for i in correct_Secondary_dict.keys():
    if i in Secondary_school_rank.keys():
        Secondary_school_rank[correct_Secondary_dict[i]]=Secondary_school_rank[i]
        del Secondary_school_rank[i]

#### Create the secondary_rank function to input the secondary school rank
        

In [ ]:
def secondary_rank(secondary_school):
    secondary_school_name = secondary_school
    # if secondary_school_name in the keys in secondary_school_rank,set the value in the dictionary of Secondary_school_rank as rank 
    if secondary_school_name in Secondary_school_rank.keys():
        rank = int(Secondary_school_rank[secondary_school_name])
    else:
        rank = 'not ranked'
    
    return rank

In [ ]:
#apply secondary_rank function to df
df['secondary_school_ranking']=df.apply(lambda x: secondary_rank(x.closest_secondary_school),axis=1)


In [ ]:
df = df[['ID','Address','Suburb','Price','Type','Date','Rooms','Bathroom','Car','Landsize','Age','Lattitude','Longtitude','train_station_id','distance_to_train_station','travel_min_to_CBD','over_priced?','crime_A_average','crime_B_average','crime_C_average','closest_primary_school','distance_to_closest_primary','primary_school_ranking','closest_secondary_school','distance_to_closest secondary','secondary_school_ranking']]
df=df.set_index('ID')

In [ ]:
df

In [ ]:
df.to_csv("Group081_solution.csv")  

# Task 2: Data Reshaping
<a id="task_2"></a>
In this task, we use different normalization and transformation methods, such as standardization, min-max normalization, log, power, and root transformation on Rooms, crime_C_acerage, travel_min_to_CBD, and property_age to find which normalization/transformation method would work better to fit the linear model on price.


In [ ]:
df = pd.read_csv('Group081_solution.csv')

In [ ]:
# extract columns used in the linear regression and save in temp
col_list = ['Price', 'Rooms', 'crime_C_average', 'Age', 'travel_min_to_CBD']
temp = df[col_list]

#### define functions to process normalisations and transformations

In [ ]:
def std_normalisation(df, col):
    std_scale = preprocessing.StandardScaler().fit(df[[col]])
    df_std = std_scale.transform(df[[col]])
    name = 'std_' + col
    df[name] = df_std[:,0]

In [ ]:
def minmax_normalisation(df, col):
    minmax_scale = preprocessing.MinMaxScaler().fit(df[[col]])
    name = 'minmax_' + col
    df[name] = minmax_scale.transform(df[[col]])

In [ ]:
def root_transformation(df, col):
    name = 'root_' + col
    df[name] = None
    i = 0
    for row in df.iterrows():
        df[name].at[i] = math.sqrt(df[col][i])
        i += 1
    df[name] = df[name].astype('float')

In [ ]:
def square_transformation(df, col):
    name = 'square_' + col
    df[name] = None
    i = 0
    for row in df.iterrows():
        df[name].at[i] = math.pow(df[col][i],2)
        i += 1
    df[name] = df[name].astype('float')

In [ ]:
def log_transformation(df, col):
    name = 'log_' + col
    df[name] = None
    i = 0
    for row in df.iterrows():
        df[name].at[i] = math.log(df[col][i])
        i += 1

    df[name] = df[name].astype('float')

In [ ]:
# show the statistical characters of the temp
temp.describe()

The values of **Price** are way larger than other values in other columns. In this situation, I suppose that **log transformation** is more suitable. And looking at the temp dataframe, it is clear that the values of log_Price column's standard error is less than others. As for the **Age** column, since the difference between min and max is large, as well as the standard error is large, using **log transformation** can solve this problem. Also due to this reason, I think log transformation is suitable for the **crime_C_average** column. As for the **travel_min_to_CBD column**, it contains the value of 0 which means it can not be applied log transformation, in this case, I assume it is used **root transformation**. When it comes to the **Rooms** column, its values are basically in a small range, so I do not think it requires any normalisation or transformation.

In [ ]:
# apply log transformation in Price, Age, crime_C_average
for col in ['Price', 'Age', 'crime_C_average']:
    log_transformation(temp, col)
    
# apply root transformation in travel_min_to_CBD
root_transformation(temp,'travel_min_to_CBD')

In [ ]:
# plot each variables in one graph
temp.log_Age.plot(color = 'red')
temp.log_crime_C_average.plot(color = 'blue')
temp.root_travel_min_to_CBD.plot(color = 'green')
temp.Rooms.plot(color = 'yellow')

In [ ]:
# for each column, plot the col - price scatter graph.
for col in ['log_Age','log_crime_C_average','root_travel_min_to_CBD','Rooms']:
    plt.scatter(temp[col], temp.log_Price)

According to the two graphs above, I think **applying log transformation in crime_C_average, Age, and Price**, **applying root transformation in travel_min_to_CBD** is a good choice.

#### Build up the model by parameters above using ols

In [ ]:
model = sm.ols(formula="log_Price ~ log_Age \
+ log_crime_C_average\
+ root_travel_min_to_CBD\
+ Rooms"\
               ,data=temp).fit() # fit the model
print(model.summary()) # show details of the model

#### plot predicted values and real values(log_Price) in one graph


In [ ]:
y_fitted = model.fittedvalues
y_fitted.plot(color='red')
temp.log_Price.plot(color='blue')

# Summary
<a id="summary"></a>

In this assignment, we learn how to integrate serveral dataets into one signle schema and fond and fix possible problems in the data by using Python 3. In addition, we use different normalization and transformation methods, such as standardization, min-max normalization, log, power, and root transformation on Rooms, crime_C_acerage, travel_min_to_CBD, and property_age to find which normalization/transformation method would work better to fit the linear model on price.